In [3]:
import pandas as pd
import numpy as np


db = pd.read_csv("/Users/home/Dropbox/Databases/fifa-world-cup/WorldCupMatches.csv")
db.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  852 non-null    float64
 1   Datetime              852 non-null    object 
 2   Stage                 852 non-null    object 
 3   Stadium               852 non-null    object 
 4   City                  852 non-null    object 
 5   Home Team Name        852 non-null    object 
 6   Home Team Goals       852 non-null    float64
 7   Away Team Goals       852 non-null    float64
 8   Away Team Name        852 non-null    object 
 9   Win conditions        852 non-null    object 
 10  Attendance            850 non-null    float64
 11  Half-time Home Goals  852 non-null    float64
 12  Half-time Away Goals  852 non-null    float64
 13  Referee               852 non-null    object 
 14  Assistant 1           852 non-null    object 
 15  Assistant 2          

In [3]:
db = db[['Year', 'City', 'Home Team Name', 'Home Team Goals',
         'Away Team Goals', 'Away Team Name', 'Half-time Home Goals',
         'Half-time Away Goals', 'Home Team Initials', 'Away Team Initials']]


# I don't this code anymore because I found a data base that has the information 
# I will still leave it here
#db.describe()
#db['Total_goals'] = db['Home Team Goals'] + db['Away Team Goals']
#db.groupby('Year')['Total_goals'].mean() # Aggregate by mean
#db.groupby('Year')['Total_goals'].sum() # Aggregate get goals
#db.groupby('Year')['Total_goals'].count() # Aggregate number of games
#db0 = (db.groupby(['Year'])['Total_goals']
 #        .agg([('average_goals','mean'),('total_goals','sum'), ('total_games','count')])
 #        .reset_index())
#db0

,Year,average_goals,total_goals,total_games
0,1930.0,3.888889,70.0,18
1,1934.0,4.117647,70.0,17
2,1938.0,4.666667,84.0,18
3,1950.0,4.000000,88.0,22
4,1954.0,5.384615,140.0,26
5,1958.0,3.600000,126.0,35
6,1962.0,2.781250,89.0,32
7,1966.0,2.781250,89.0,32
8,1970.0,2.968750,95.0,32
9,1974.0,2.552632,97.0,38


In [13]:
# Aggregated information from world cups
#db3 = pd.read_csv("/Users/home/Dropbox/Databases/fifa-world-cup/WorldCups.csv")
# World cup 2018
db2 = pd.read_csv("/Users/home/Dropbox/Databases/fifa-world-cup/Cup.Russia.Matches.csv")

db2 = db2[['Home Team',
           'Home Team Goals',  'Away Team Goals',
           'Away Team', 'Total Goals', 'Attendance']]

db2['Year'] = 2018
db2['Country'] = 'Russia'
db2['GoalsScored'] = db2['Total Goals'].sum()
# Number of  Qualified teams

# Extract useful columns
db2_1 = db2['Home Team']
db2_2 = db2['Away Team']

# Rename columns to merge
db2_1.columns = ['Teams']
db2_2.columns = ['Teams']

data_merging = [db2_1, db2_2]
db2_12 = pd.concat(data_merging)

# Extract only unique cases
number = len(db2_12.unique().tolist())
db2['Qualifiedteams'] =  number

db2['MatchesPlayed'] = db2['Home Team'].count()
db2['Total_attendance'] = db2['Attendance'].sum()
db2.head()


#Winner	
#Runners-Up	
#Third
#Fourth

# Merge data sets

#db_final = pd.merge(db, db3, how='inner', left_on='Year', right_on='Year')
#db_final.info()

,Home Team,Home Team Goals,Away Team Goals,Away Team,Total Goals,Attendance,Year,Country,GoalsScored,Qualifiedteams,MatchesPlayed,Total_attendance
0,Russia,5,0,Saudi Arabia,5,78011,2018,Russia,169,32,64,3031766
1,Egypt,0,1,Uruguay,1,27015,2018,Russia,169,32,64,3031766
2,Morocco,0,1,IR Iran,1,62548,2018,Russia,169,32,64,3031766
3,Portugal,3,3,Spain,6,43866,2018,Russia,169,32,64,3031766
4,France,2,1,Australia,3,41279,2018,Russia,169,32,64,3031766


32